In [72]:
import numpy as np
import openrouteservice
from openrouteservice import convert
from datetime import datetime, timedelta 
from random import randint
import json
from datetime import datetime, timedelta
import folium
from folium import plugins
from folium.plugins import FloatImage
import os
import matplotlib.pyplot as plt
import matplotlib as mpl

# PARAMETRI DI INPUT

n_users = 6           # MAX: 12. Quanti utenti includere nella simulazione
delta_dist = 0.0035     # Raggio della circonferenza che contiene i 
                      # punti di partenza (in gradi decimali)
delta_grid = 0.01

dpi_list = ["mascherina", "guanti", "guanti e mascherina", "nessuno"]
simptoms_list = ["emicrania", "nausea", "dolori articolari", "respiro affannoso"]

# PUNTI DI INTERESSE A TERNI
#interest_points = [{"name_loc": "CONAD Superstore", "addr": "Via del Rivo, 206, 05100 Terni TR, Italia", "lat":42.580256, "lon":12.621649, "cap": "05100", "type":"acquisti"},
#                   {"name_loc": "Farmacia Nadalini", "addr": "Via del Rivo, 98-80, 05100 Terni TR, Italia", "lat":42.579970, "lon":12.625367, "cap": "05100", "type":"acquisti"},
#                   {"name_loc": "Unicredit", "addr": "Via del Rivo, 208, 05100 Terni TR, Italia", "lat":42.580893, "lon":12.621879, "cap": "05100", "type":"banca"},
#                   {"name_loc": "Ufficio", "addr": "Via del Camoscio, 19, 05100 Terni TR, Italia", "lat":42.582965, "lon":12.619052, "cap": "05100", "type":"ufficio"},
#                   {"name_loc": "Ambulatorio", "addr": "Via del Rivo, 93, 05100 Terni TR, Italia", "lat":42.579069, "lon":12.626082, "cap": "05100", "type":"ambulatorio"}]
#center_coords = (42.581302, 12.622702)

# PUNTI DI INTERESSE AD AMSTERDAM
interest_points = [{"name_loc": "Name", "addr": "Addr", "lat":51.892337, "lon":4.504295, "cap": "cap", "type":"type"},
                   {"name_loc": "Name", "addr": "Addr", "lat":51.891287, "lon":4.501049, "cap": "cap", "type":"type"},
                   {"name_loc": "Name", "addr": "Addr", "lat":51.892822, "lon":4.496836, "cap": "cap", "type":"type"},
                   {"name_loc": "Name", "addr": "Addr", "lat":51.894476, "lon":4.492928, "cap": "cap", "type":"type"},
                   {"name_loc": "Name", "addr": "Addr", "lat":51.896909, "lon":4.497276, "cap": "cap", "type":"type"}]

center_coords = (51.893340, 4.501354)

# Calcolo vertici griglia
upper_right = (center_coords[0]+delta_grid,center_coords[1]+delta_grid*1.5) 
lower_left = (center_coords[0]-delta_grid,center_coords[1]-delta_grid*1.5) 

# Variabili per l'animazione
# I colori sono dello stesso numero di n_users
colors = ["green", "blue", "red", "purple", "orange", "brown", "violet", 
          "crimson", "lightgreen", "grey", "lightblue", "lightgreen"]
segments_opacity = 1
segments_weight = 4
delta_coord = 0.000002

# FUNZIONI GENERATORE
def get_itinerary(trip):
    """
    This function takes a tuple of GPS positions tuples (lat, lon) and 
    returns openstreetmaps directions, in 
    terms of gps coordinates array.
    """
    # Change your token here
    token = '5b3ce3597851110001cf62488a48a7f1e3ad491184353a3b070c32a2'
    
    # Connect to client
    client = openrouteservice.Client(key=token) # Specify your token

    # decode_polyline needs the geometry only
    geometry = client.directions(trip)['routes'][0]['geometry']
    decoded = convert.decode_polyline(geometry)

    return decoded['coordinates']

# FUNZIONI ANIMAZIONE
def create_segments(input_trip, color, trip_number):

    segments_single_user = [] # Analogo di "lines" nell'esempio

    start_time_str = input_trip["info"]["start time"]

    start_time = datetime.strptime(start_time_str, '%d-%m-%YT%H:%M:%S')

    # Acquisisco coordinate GPS
    coord_list = input_trip["directions"]
    n_positions = len(coord_list)

    # Crea la lista di timestamp per ogni coordinata
    timestamp_list = []
    timestamp_list.append(start_time.strftime("%Y-%m-%dT%H:%M:%S"))
    prev_time = start_time

    for idx in range(0, n_positions):
    
        this_time = prev_time + timedelta(minutes = 1)
        timestamp_list.append(this_time.strftime("%Y-%m-%dT%H:%M:%S"))
        prev_time = this_time

    # Ora va creato il dict, uno per ogni segmento, da appendere all'array "segments_single_user"
    for idx in range(1, n_positions):

        #increment = 0
        increment = (trip_number+1)*delta_coord
        
        single_line = {}
        single_line["coordinates"] = [[coord_list[idx-1][0]+increment,coord_list[idx-1][1]+increment],
                                      [coord_list[idx][0]+increment,coord_list[idx][1]+increment]]
                                          
        
        
        single_line["dates"] = [timestamp_list[idx-1],timestamp_list[idx],]
        single_line["color"] = color
        single_line["weight"] = segments_weight
    
        segments_single_user.append(single_line)
    
    #for item in segments_single_user:
    #    print(item)
    
    return segments_single_user

def create_points(input_trip, color, trip_number):
    
    points_single_user_list = []
    
    point_single_user_1 = {}
    point_single_user_1["coordinates"] = [input_trip[0]["coordinates"][0][0],input_trip[0]["coordinates"][0][1]]
    point_single_user_1["date"] = input_trip[0]["dates"][0]
    point_single_user_1["color"] = color
    point_single_user_1["icon"] = "none"
    points_single_user_list.append(point_single_user_1)
    
    point_single_user_2 = {}
    point_single_user_2["coordinates"] = [input_trip[-1]["coordinates"][1][0],input_trip[-1]["coordinates"][1][1]]
    point_single_user_2["date"] = input_trip[-1]["dates"][-1]
    point_single_user_2["color"] = color
    point_single_user_2["icon"] = "circle"
    points_single_user_list.append(point_single_user_2)
    
    return points_single_user_list

def prepare_segment_features(segments_list):

    segment_features = [
        {
            'type': 'Feature',
            'geometry': {
                'type': 'LineString',
                'coordinates': segment['coordinates'],
            },
            'properties': {
                'times': segment['dates'],
                'style': {
                    'color': segment['color'],
                    'opacity': segments_opacity,
                    'weight': segment['weight'] if 'weight' in segment else 5
                }
            }
        }
        for segment in segments_list
    ]    
    
    return segment_features

def prepare_point_features(all_points_list):
    
    #print(all_points_list[0]['icon'])
    
    point_features = [
        {
            'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates': point['coordinates'],
            },
            'properties': {
                'times': [point['date']],
                'icon': point['icon'], 
                'iconstyle': {'color': point['color'], 'radius': 15},

            }
        }
        for point in all_points_list
    ]
    
    return point_features

# INIZIO SCRIPT

lats = list(np.random.uniform(center_coords[0]-delta_dist, center_coords[0]+delta_dist, n_users))
lons = list(np.random.uniform(center_coords[1]-delta_dist, center_coords[1]+delta_dist, n_users))
uuids = list(np.random.randint(low = 1000, high = 2000, size=n_users))

starting_points = [list(a) for a in zip(uuids, lats, lons)]

# GENERAZIONE TRIP
raw_trips = []

for point in starting_points:

    trip = {}
    info = {}
    
    # Scelta destinazione per uuid corrente
    n = np.random.randint(len(interest_points))
    
    trip["uuid"] = int(point[0])
    trip["lat start"] = float(point[1])
    trip["lon start"] = float(point[2])
    trip["lat dest"] = interest_points[n]["lat"]
    trip["lon dest"] = interest_points[n]["lon"]
    trip["name dest"] = interest_points[n]["name_loc"]
    trip["address dest"] = interest_points[n]["addr"]
    trip["cap dest"] = interest_points[n]["cap"]
    trip["type dest"] = interest_points[n]["type"]
    
    # Generazione itinerario
    start_loc = (trip["lon start"], trip["lat start"])
    dest_loc = (trip["lon dest"], trip["lat dest"])
    trip_tuple = (start_loc, dest_loc)
    
    itinerary = get_itinerary(trip_tuple)
    trip["directions"] = list(itinerary)
    
    # Parametri utente
    temperature = np.random.uniform(35.5,37.5)
    simptoms = simptoms_list[np.random.randint(len(simptoms_list))]
    dpi = dpi_list[np.random.randint(len(dpi_list))]
    
    info["temperature"] = temperature
    info["simptoms"] = simptoms
    info["dpi"] = dpi
    
    start_time = datetime.now()
    
    # Add random time delta
    delta_h = 0
    delta_min = 5
    factor = randint(0, 6)
    start_time = start_time + timedelta(hours = delta_h*factor, minutes = delta_min*factor)
    
    info["start time"] = start_time.strftime("%d-%m-%YT%H:%M:%S") 
    
    trip["info"] = info

    raw_trips.append(trip)
    
# GENERAZIONE GEOJSON
#trip_list = []

#for item in raw_trips:
    
#    json_trip = json.dumps(item)
#    trip_list.append(json_trip)
#    print(json_trip)

trip_list = raw_trips

# INIZIO PARTE DI ANIMAZIONE

# Creo tutti i segmenti per tutti i trip e crea un'unica features list
all_segments_list = []
all_points_list = []

idx = 0
for single_trip in trip_list:
    
    segments_list = create_segments(single_trip, colors[idx], idx)
    all_segments_list = all_segments_list + segments_list

    points_list = create_points(segments_list, colors[idx], idx)
    all_points_list = all_points_list + points_list
    
    idx = idx + 1
    
# Creazione mappa reale

m = folium.Map(
    location=center_coords,
    #tiles="stamenterrain",
    #tiles="stamentoner",
    #tiles="stamenwatercolor",
    zoom_start=16
)

# Creo le features
segment_features = prepare_segment_features(all_segments_list)
point_features = prepare_point_features(all_points_list)
#print(point_features[0])
#print(segment_features[0])

features = segment_features + point_features

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT1M',duration='P1D',auto_play=False,loop=True,loop_button=True,add_last_point=False).add_to(m)

FloatImage("logo_whitelabel.jpg", bottom=10, left=3).add_to(m)

m.save("simulation_real.html")
m

In [73]:
# Creazione mappa sintetica con griglia

# FUNZIONI PER GRIGLIA
def get_geojson_grid(upper_right, lower_left, n=6):
    
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.

    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).

    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).

    n: integer
        The number of rows/columns in the (n,n) grid.

    Returns
    -------

    list
        List of "geojson style" dictionary objects   
    """

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)

    return all_boxes

# SECONDA PARTE SCRIPT

m = folium.Map(
    location=center_coords,
    #tiles="stamenterrain",
    #tiles="stamentoner",
    #tiles="stamenwatercolor",
    zoom_start=16
)

# Creo le features
segment_features = prepare_segment_features(all_segments_list)
point_features = prepare_point_features(all_points_list)
#print(point_features[0])
#print(segment_features[0])

features = segment_features + point_features

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT1M',duration='P1D',auto_play=False,loop=True,loop_button=True,add_last_point=False).add_to(m)

FloatImage("logo_whitelabel.jpg", bottom=10, left=3).add_to(m)

# Crea e append griglia
grid = get_geojson_grid(upper_right, lower_left , n=15)

for i, geo_json in enumerate(grid):
    
    # Ogni loop di questo for è una cella
    #print(i)

    color = plt.cm.Blues(6 / len(grid))
    color = mpl.colors.to_hex(color)

    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"blue",
                                                                        'weight': 0.5,
                                                                         #'dashArray': '5, 5',
                                                                        'fillOpacity': 0.15,
                                                                    })
    popup = folium.Popup("Cell n.{}".format(i+1))
    gj.add_child(popup)

    m.add_child(gj)

# Parte per tile vuota
folium.TileLayer('Mapbox Control Room').add_to(m)
folium.LayerControl().add_to(m)

m.save("simulation_grid.html")
m